# DSSA 
## Robert Tarasevič TSF–17

## 3. lab. darbas | Logtiesiniai modeliai su R                                  


Duomenų faile ksimpah.csv yra realaus medicininio tyrimo duomenys apie paauglių
susirgimą pirminę arterinę hipertenziją. Kategoriniai dvireikšmiai kintamieji:

- H Hipertenzija -- ar pacientas serga pirmine arterine hipertenzija;
- L Lytis -- paciento lytis;
- T Tabakas -- nurodo, ar pacientas rūko;
- A Alkoholis -- nurodo, ar pacientas vartoja alkoholį;
- S Antsvoris -- nurodo, ar pacientas turi antsvorį.


Sudarykite įvairias dažnių lenteles ir, naudojant logistinės regresijos 
modelius, atlikite užduotis.

1.  Sudarykite trijų kintamųjų T, A ir S kryžminę dažnių lentelę. Sudarykite 
    tokį logtiesinį modelį, kad, naudojant logtikėtinumo santykio testą, būtų 
    galima patikrinti hipotezę, kad šie kintamieji yra nepriklausomi.<br>

2. Sudarykite tokį logtiesinį modelį, kad būtų galima patikrinti hipotezę,
    jog kintamųjų T ir S sąveika T:S yra reikšminga, o kitos dvi sąveikos
    nereikšmingos. <br>

3. Sudarykite tokį logtiesinį modelį, kad būtų galima patikrinti hipotezę,
    kad T ir A yra sąlyginai nepriklausomi, kai žinomas S. Tai reiškia, kad
    T ir A galbūt yra priklausomi, bet atskirai turinčių antsvorį grupėje
    nepriklausomi ir atskirai neturinčių antsvorio grupėje taip pat yra
    nepriklausomi.
    Čia naudingas skyrelis 5.2.5 Partial Independence iš konspekto c5.pdf <br>

4. Apskaičiuokite dažnių prognozes, kai modelyje yra nėra visų trijų T, A 
    ir S porų sąveikų ir kai jos yra. Nustatykite, kokiu atveju prognozė 
    geresnė: kai modelyje yra sąveika T su A ar kai modelyje yra sąveika 
    T su S? Dažnių prognozei naudojama ta pati standartinė funkcija <code>fitted</code> 
    arba <code>predict.glm</code>.


Nuskaitome duomenis.

In [1]:
d = read.csv2("ksimpah.csv", encoding = 'UTF-8')
head(d)

,Lytis,Hipertenzija,Tabakas,Alkoholis,Ūgis,Svoris,KMI,Antsvoris,EQ,EZ,FC,FE
,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,Vyras,Sveikas,Nerūko,Geria,181.0,77.0,23.50,Ne,519.5,803.0,1.20,4.62
2,Vyras,Sveikas,Nerūko,Geria,182.0,108.0,32.60,Yra,517.5,797.0,2.15,5.22
3,Vyras,Serga,Nerūko,Negeria,187.0,122.0,34.89,Yra,654.0,554.5,2.85,4.89
4,Vyras,Sveikas,Nerūko,Geria,178.5,64.0,20.09,Ne,518.5,727.5,1.40,4.56
5,Moteris,Sveikas,Rūko,Geria,170.0,57.5,19.90,Ne,472.5,662.5,1.80,4.01
6,Vyras,Sveikas,Nerūko,Geria,187.5,70.0,19.91,Ne,516.0,680.5,1.55,4.73


### 1. Sudarykite trijų kintamųjų T, A ir S kryžminę dažnių lentelę. Sudarykite tokį logtiesinį modelį, kad, naudojant logtikėtinumo santykio testą, būtų galima patikrinti hipotezę, kad šie kintamieji yra nepriklausomi.
***

Kryžminei lentelei sudaryti naudosime <code>xtabs</code> funkciją, toliau mums reikės sudaryti tokius du *glm* modelius, kad viename būtų kintamųjų sąveika, o kitame ne. Toliau taikysime *anova* funkciją santykiniam testui atlikti, kad patikrinti kintamūju nepriklausomumą.

#### T -- tabakas
#### S -- antsvoris
#### A -- alkoholio vartojimas
#### F -- dažniai


In [2]:
l = xtabs( ~ 1 + Tabakas + Alkoholis + Antsvoris, data=d)
l = ftable(l)
N = as.data.frame(l)
names(N) <- c("T", "A", "S", "F")
N

T,A,S,F
<fct>,<fct>,<fct>,<int>
Nerūko,Geria,Ne,39
Rūko,Geria,Ne,54
Nerūko,Negeria,Ne,23
Rūko,Negeria,Ne,1
Nerūko,Geria,Yra,7
Rūko,Geria,Yra,17
Nerūko,Negeria,Yra,11
Rūko,Negeria,Yra,1


In [3]:
mod1 = glm(F ~ 1 + T + A + S, N, family = poisson)
mod2 = glm(F ~ 1 + T + A, N, family = poisson)

anova(mod1, mod2, test="Chisq")

,Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,43.49905,NA,NA,NA
2,5,88.65015,-1,-45.1511,1.824034e-11


Pagal p reikšmę = 1.82e-11 matome, kad hipotezė $H_0$ apie parametrų nereikšmingumą yra atmetamą, *Antsvorio* parametras yra reikšmingas.

### 2. Sudarykite tokį logtiesinį modelį, kad būtų galima patikrinti hipotezę, jog kintamųjų T ir S sąveika T:S yra reikšminga, o kitos dvi sąveikos nereikšmingos. 

Sudarome tokie modelį, kuriame būtų visos 3 sąveikos:
- T:S
- T:A
- S:A

In [4]:
mod3 = glm(F ~ 1 + T + A + S + T:A + T:S + S:A, data = N, family=poisson)
summary(mod3)


Call:
glm(formula = F ~ 1 + T + A + S + T:A + T:S + S:A, family = poisson, 
    data = N)

Deviance Residuals: 
       1         2         3         4         5         6         7         8  
-0.01179   0.01003   0.01538  -0.07196   0.02795  -0.01785  -0.02217   0.07559  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      3.6654     0.1591  23.036  < 2e-16 ***
TRūko            0.3222     0.2081   1.548   0.1217    
ANegeria        -0.5332     0.2592  -2.057   0.0397 *  
SYra            -1.7301     0.3974  -4.354 1.34e-05 ***
TRūko:ANegeria  -3.3833     0.7648  -4.424 9.69e-06 ***
TRūko:SYra       0.5800     0.4707   1.232   0.2178    
ANegeria:SYra    1.0024     0.5151   1.946   0.0516 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 134.121619  on 7  degrees of freedom
Residual deviance:   0.012961  on 1  degrees of freedom
AIC: 47.069

Number of Fi

Pagal sudaryti modelio <code>summary</code> matome:

* T:A sąveika yra labai reikšminga (p=9,69e-6)
* T:S sąveika nereikšminga (p=0,2178)
* S:A sąveika yra nereikšminga bet ir tuo pačiu "ant ribos" (p=0.0516), tad su atitinkamu reikšmingumo lygmeniu ši sąveika galėtų būti reikšminga.

### 3. Sudarykite tokį logtiesinį modelį, kad būtų galima patikrinti hipotezę, kad T ir A yra sąlyginai nepriklausomi, kai žinomas S. Tai reiškia, kad T ir A galbūt yra priklausomi, bet atskirai turinčių antsvorį grupėje nepriklausomi ir atskirai neturinčių antsvorio grupėje taip pat yra nepriklausomi. Čia naudingas skyrelis 5.2.5 Partial Independence iš konspekto c5.pdf 

Sudarome du modelius:

* sat: 1 + S:T + S:A
* mod: 1 + S

In [5]:
sat = glm(F ~ 1 + S:T + S:A, N, family=poisson)
mod = glm(F ~ 1 + S, N, family=poisson)

Pasinaudoję <code>anova</code> funkcija atliksime tikėtinumo santykio testą

In [6]:
anova(mod, sat, test = "Chisq")

,Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6,88.97052,NA,NA,NA
2,2,41.01606,4,47.95446,9.646409e-10


Iš testo rezultatų lentelės matome, kad S:T ir S:A sankirtos yra statistiškai reikšmingos.

#### 4. Apskaičiuokite dažnių prognozes, kai modelyje yra nėra visų trijų T, A ir S porų sąveikų ir kai jos yra. Nustatykite, kokiu atveju prognozė geresnė: kai modelyje yra sąveika T su A ar kai modelyje yra sąveika T su S? 
#### Dažnių prognozei naudojama ta pati standartinė funkcija <code>fitted</code>  arba <code>predict.glm</code>.

Kaip sąlyga ir prašo, sudarome du modelius:

1. modelis turintis visų trijų **T,A** ir **S** porų sąveikas;
2. modelis neturintis kintamųjų sąveikų.

Sudarę šiuos modelius, palyginsime jų prognozes ir nustatysime kuriuo atvėju prognozė yra geresnė. Dažnių prognozei atlikti naudosime <code>fitted</code> funkciją.

In [7]:
mod1 = glm(F ~ 1 + T + A + S + T:A + T:S + S:A, data = N, family = poisson)
mod2 = glm(F ~ 1 + T + A + S, data = N, family = poisson)

pred_1 = matrix(fitted(mod1))
pred_2 = matrix(fitted(mod2))

pred = as.data.frame(cbind(pred_1, pred_2, N$F))
names(pred) <- c("Modelis su sąveikom", "Modelis neturintis sąveikų", "Tikrosios reikšmės")

pred


Modelis su sąveikom,Modelis neturintis sąveikų,Tikrosios reikšmės
<dbl>,<dbl>,<dbl>
39.0736997,46.782007,39
53.9263003,42.688581,54
22.9263003,14.394464,23
1.0736997,13.134948,1
6.9263003,14.394464,7
17.0736997,13.134948,17
11.0736997,4.429066,11
0.9263003,4.041522,1


Pasižiūrėjus į dažnių lentelę, nesunku pastebėti, kad pirmasis modelis (turintis kintamųjų sąveikas) prognozuoja gerokai tiksliau nei modelis neturintis kintamųjų sąveikas.